In [1]:
import pandas

In [2]:
data_file = pandas.ExcelFile('./src/real data/2017 CAM data from iPads/2017 CAM data from iPads.xlsx')

In [3]:
data_file.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

In [4]:
# read the data from one of the sheets into a dataframe
df = data_file.parse('2017 CAM data Erl')

In [5]:
# what's the general structure of the sheet?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 50 columns):
fields__client__company                                             8 non-null object
fields__client__name                                                8 non-null object
fields__client__fname                                               8 non-null object
fields__client__lname                                               8 non-null object
fields__client__displayText                                         8 non-null object
fields__name                                                        8 non-null object
fields__crop                                                        8 non-null object
fields__desc                                                        7 non-null object
fields__image                                                       8 non-null object
fields__date                                                        8 non-null object
fields__oSets__date                

In [6]:
# Which dates are in this dataset?
pandas.Series(df.fields__oSets__date.dropna().sort_values().unique())

0    2017-08-02T13:12:09.542
1    2017-08-09T09:25:11.710
2    2017-08-09T10:06:25.480
3    2017-08-09T11:21:01.555
4    2017-08-09T11:37:20.862
5    2017-08-17T11:12:02.820
6    2017-08-17T13:06:30.183
7    2017-08-22T15:42:05.751
8    2017-08-22T16:02:50.682
dtype: object

In [7]:
# make the really long column names easier to reference
lat_col, long_col = ('fields__oSets__oPoints__location__coords__latitude',
                     'fields__oSets__oPoints__location__coords__longitude')

# pick unique rows with extreme coordinates, just for the sake of variety
locations = df.loc[[df[lat_col].idxmax(),
                    df[long_col].idxmin(),
                    df[lat_col].idxmin(),
                    df[long_col].idxmax(),
                   ]].drop_duplicates()

# focus on just the latitude and longitude columns
map_centers = locations[[lat_col, long_col]]

display(map_centers)

,fields__oSets__oPoints__location__coords__latitude,fields__oSets__oPoints__location__coords__longitude
350,52.155109,-106.728488
84,52.134350,-106.634421
490,52.143679,-106.543992


In [8]:
from ipyleaflet import Map

# display a map for each coordinate set
for index, map_center in map_centers.iterrows():
    coordinates = map_center.values.tolist()
    display(df[['fields__name', 'fields__crop', 'fields__desc']].loc[index],
            Map(center=coordinates, zoom=13))

fields__name                              SEF Oats
fields__crop                            CROPS.OATS
fields__desc    Next to SEF peas and a wheat field
Name: 350, dtype: object

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

fields__name    NaN
fields__crop    NaN
fields__desc    NaN
Name: 84, dtype: object

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

fields__name         Kernen wheat
fields__crop          CROPS.WHEAT
fields__desc    Kernen cover crop
Name: 490, dtype: object

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …